In [348]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sys, os
from PIL import Image

In [233]:
# base path
train_base_path = os.path.join(os.curdir, "Train")
test_base_path = os.path.join(os.curdir, "Test")

# 火(Fire) path
fire = "火"
fire_train_base_path = os.path.join(train_base_path, fire)
fire_test_base_path = os.path.join(test_base_path, fire)

# 水(Water) path
water = "水"
water_train_base_path = os.path.join(train_base_path, water)
water_test_base_path = os.path.join(test_base_path, water)

fires_train = os.listdir(fire_train_base_path)
fires_test = os.listdir(fire_test_base_path)
waters_train = os.listdir(water_train_base_path)
waters_test = os.listdir(water_test_base_path)

fire_paths = []
water_paths = []

for fire in fires_train:
    fire_paths.append(os.path.join(fire_train_base_path, fire))
for fire in fires_test:
    fire_paths.append(os.path.join(fire_test_base_path, fire))
for water in waters_train:
    water_paths.append(os.path.join(water_train_base_path, water))
for water in waters_test:
    water_paths.append(os.path.join(water_test_base_path, water))

image_paths = fire_paths + water_paths

np.random.seed(7)
np.random.shuffle(image_paths)

In [234]:
train_paths = image_paths[:1281]
test_paths = image_paths[1281:]

In [354]:
def preprocess(image_path, input_size):
    image_path = image_path.numpy().decode("utf-8")
    image = Image.open(image_path)
    image = image.convert('L')
    image_word = (image_path.split("\\")[-2][0]).split("\\")[0]
    if image_word == "火":
        y = tf.constant(1.0)
    else:
        y = tf.constant(0.0)
    image = image.resize((input_size, input_size))
    image = np.array(image)
    image = image / 255
    image = np.reshape(image, (input_size*input_size))
    image = tf.cast(image, dtype=tf.float32)
    return image, y

In [355]:
def get_dataset(image_paths, input_size, shuffle_buffer_size=10000, n_parse_threads=5, 
                batch_size=32, cache=True):
    dataset = tf.data.Dataset.list_files(image_paths)
    dataset = dataset.map(lambda image_path: tf.py_function(preprocess, [image_path, input_size], 
                                                            [tf.float32, tf.float32]), 
                          num_parallel_calls=n_parse_threads
    )
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    def _fixup_shape(images, labels):
        images.set_shape([input_size*input_size]) 
        labels.set_shape([]) 
        return images, labels
    dataset = dataset.map(_fixup_shape)
    if cache:
        dataset = dataset.cache()
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [356]:
def train_linear_model(input_size, train_set, valid_set):
    seed = 7
    np.random.seed(seed)
    tf.random.set_seed(seed)
    input1 = keras.layers.Input(shape=(input_size*input_size))
    output = keras.layers.Dense(1, activation='sigmoid')(input1)
    linear_model = keras.Model(inputs=[input1], outputs=[output])
    linear_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    batch_size = 32
    checkpoint_path = "xinshuai_models/linearmodel/"+str(input_size)+"/{epoch:04d}.ckpt"
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        monitor='val_accuracy',
        filepath=checkpoint_path, 
        verbose=1, 
        mode="max",
        save_weights_only=True,
        save_freq="epoch",
        save_best_only=True
    )
    linear_model.save_weights(checkpoint_path.format(epoch=0))
    earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=70, verbose=1)
    linear_model.fit(train_set, validation_data=valid_set, epochs=10000, batch_size=batch_size, 
                 callbacks=[cp_callback, earlystop_callback])
    return linear_model

In [357]:
input_size = 8
train_set = get_dataset(train_paths, input_size)
valid_set = get_dataset(test_paths, input_size)

In [358]:
linear_model = train_linear_model(input_size, train_set, valid_set)

Epoch 1/10000
 1/41 [..............................] - ETA: 0s - loss: 0.7422 - accuracy: 0.4688
Epoch 00001: val_accuracy improved from -inf to 0.49000, saving model to xinshuai_models/linearmodel/8\0001.ckpt
41/41 [==============================] - 0s 6ms/step - loss: 0.7080 - accuracy: 0.4910 - val_loss: 0.7045 - val_accuracy: 0.4900
Epoch 2/10000
 1/41 [..............................] - ETA: 0s - loss: 0.7162 - accuracy: 0.4062
Epoch 00002: val_accuracy did not improve from 0.49000
41/41 [==============================] - 0s 1ms/step - loss: 0.7001 - accuracy: 0.4965 - val_loss: 0.7004 - val_accuracy: 0.4550
Epoch 3/10000
 1/41 [..............................] - ETA: 0s - loss: 0.6938 - accuracy: 0.4375
Epoch 00003: val_accuracy did not improve from 0.49000
41/41 [==============================] - 0s 1ms/step - loss: 0.6937 - accuracy: 0.4895 - val_loss: 0.6972 - val_accuracy: 0.4600
Epoch 4/10000
 1/41 [..............................] - ETA: 0s - loss: 0.7019 - accuracy: 0.5000
Ep

Epoch 28/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5744 - accuracy: 0.9375
Epoch 00028: val_accuracy did not improve from 0.82500
41/41 [==============================] - 0s 1ms/step - loss: 0.6133 - accuracy: 0.8212 - val_loss: 0.6135 - val_accuracy: 0.8050
Epoch 29/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5950 - accuracy: 0.8438
Epoch 00029: val_accuracy did not improve from 0.82500
41/41 [==============================] - 0s 1ms/step - loss: 0.6101 - accuracy: 0.7830 - val_loss: 0.6134 - val_accuracy: 0.7650
Epoch 30/10000
 1/41 [..............................] - ETA: 0s - loss: 0.6093 - accuracy: 0.8438
Epoch 00030: val_accuracy improved from 0.82500 to 0.83500, saving model to xinshuai_models/linearmodel/8\0030.ckpt
41/41 [==============================] - 0s 2ms/step - loss: 0.6106 - accuracy: 0.7642 - val_loss: 0.6076 - val_accuracy: 0.8350
Epoch 31/10000
 1/41 [..............................] - ETA: 0s - loss: 0.6140 - accuracy: 0.

Epoch 56/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5183 - accuracy: 0.9062
Epoch 00056: val_accuracy did not improve from 0.87500
41/41 [==============================] - 0s 1ms/step - loss: 0.5483 - accuracy: 0.8774 - val_loss: 0.5451 - val_accuracy: 0.8600
Epoch 57/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5193 - accuracy: 0.9688
Epoch 00057: val_accuracy did not improve from 0.87500
41/41 [==============================] - 0s 1ms/step - loss: 0.5462 - accuracy: 0.8790 - val_loss: 0.5431 - val_accuracy: 0.8550
Epoch 58/10000
 1/41 [..............................] - ETA: 0s - loss: 0.6043 - accuracy: 0.8125
Epoch 00058: val_accuracy did not improve from 0.87500
41/41 [==============================] - 0s 1ms/step - loss: 0.5443 - accuracy: 0.8720 - val_loss: 0.5410 - val_accuracy: 0.8650
Epoch 59/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5240 - accuracy: 0.9062
Epoch 00059: val_accuracy did not improve from 0.87500
4

Epoch 85/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5231 - accuracy: 0.8438
Epoch 00085: val_accuracy did not improve from 0.89000
41/41 [==============================] - 0s 1ms/step - loss: 0.4990 - accuracy: 0.8852 - val_loss: 0.4935 - val_accuracy: 0.8750
Epoch 86/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5110 - accuracy: 0.9062
Epoch 00086: val_accuracy did not improve from 0.89000
41/41 [==============================] - 0s 2ms/step - loss: 0.4975 - accuracy: 0.8806 - val_loss: 0.4920 - val_accuracy: 0.8600
Epoch 87/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5132 - accuracy: 0.8750
Epoch 00087: val_accuracy did not improve from 0.89000
41/41 [==============================] - 0s 1ms/step - loss: 0.4961 - accuracy: 0.8891 - val_loss: 0.4905 - val_accuracy: 0.8600
Epoch 88/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5084 - accuracy: 0.8750
Epoch 00088: val_accuracy did not improve from 0.89000
4

Epoch 114/10000
 1/41 [..............................] - ETA: 0s - loss: 0.4533 - accuracy: 0.9688
Epoch 00114: val_accuracy did not improve from 0.89500
41/41 [==============================] - 0s 1ms/step - loss: 0.4616 - accuracy: 0.8946 - val_loss: 0.4540 - val_accuracy: 0.8650
Epoch 115/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5205 - accuracy: 0.8438
Epoch 00115: val_accuracy did not improve from 0.89500
41/41 [==============================] - 0s 1ms/step - loss: 0.4603 - accuracy: 0.8970 - val_loss: 0.4523 - val_accuracy: 0.8850
Epoch 116/10000
 1/41 [..............................] - ETA: 0s - loss: 0.4980 - accuracy: 0.8438
Epoch 00116: val_accuracy did not improve from 0.89500
41/41 [==============================] - 0s 1ms/step - loss: 0.4591 - accuracy: 0.8938 - val_loss: 0.4512 - val_accuracy: 0.8700
Epoch 117/10000
 1/41 [..............................] - ETA: 0s - loss: 0.4415 - accuracy: 0.8750
Epoch 00117: val_accuracy did not improve from 0.895

Epoch 143/10000
 1/41 [..............................] - ETA: 0s - loss: 0.4161 - accuracy: 0.9688
Epoch 00143: val_accuracy did not improve from 0.90000
41/41 [==============================] - 0s 1ms/step - loss: 0.4332 - accuracy: 0.8915 - val_loss: 0.4232 - val_accuracy: 0.8950
Epoch 144/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3889 - accuracy: 0.9688
Epoch 00144: val_accuracy did not improve from 0.90000
41/41 [==============================] - 0s 1ms/step - loss: 0.4322 - accuracy: 0.8977 - val_loss: 0.4215 - val_accuracy: 0.8750
Epoch 145/10000
 1/41 [..............................] - ETA: 0s - loss: 0.4238 - accuracy: 0.9062
Epoch 00145: val_accuracy did not improve from 0.90000
41/41 [==============================] - 0s 1ms/step - loss: 0.4303 - accuracy: 0.8993 - val_loss: 0.4211 - val_accuracy: 0.8900
Epoch 146/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3953 - accuracy: 0.9062
Epoch 00146: val_accuracy did not improve from 0.900

Epoch 172/10000
 1/41 [..............................] - ETA: 0s - loss: 0.4323 - accuracy: 0.8438
Epoch 00172: val_accuracy did not improve from 0.90000
41/41 [==============================] - 0s 1ms/step - loss: 0.4102 - accuracy: 0.8923 - val_loss: 0.3977 - val_accuracy: 0.8900
Epoch 173/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3912 - accuracy: 0.9062
Epoch 00173: val_accuracy did not improve from 0.90000
41/41 [==============================] - 0s 1ms/step - loss: 0.4093 - accuracy: 0.9048 - val_loss: 0.3972 - val_accuracy: 0.8950
Epoch 174/10000
 1/41 [..............................] - ETA: 0s - loss: 0.4108 - accuracy: 0.8750
Epoch 00174: val_accuracy did not improve from 0.90000
41/41 [==============================] - 0s 1ms/step - loss: 0.4071 - accuracy: 0.8962 - val_loss: 0.3962 - val_accuracy: 0.8900
Epoch 175/10000
 1/41 [..............................] - ETA: 0s - loss: 0.4141 - accuracy: 1.0000
Epoch 00175: val_accuracy did not improve from 0.900

 1/41 [..............................] - ETA: 0s - loss: 0.3490 - accuracy: 0.9375
Epoch 00229: val_accuracy did not improve from 0.91000
41/41 [==============================] - 0s 972us/step - loss: 0.3747 - accuracy: 0.8970 - val_loss: 0.3611 - val_accuracy: 0.9100
Epoch 230/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3609 - accuracy: 0.9688
Epoch 00230: val_accuracy did not improve from 0.91000
41/41 [==============================] - 0s 1ms/step - loss: 0.3729 - accuracy: 0.9040 - val_loss: 0.3608 - val_accuracy: 0.8950
Epoch 231/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3775 - accuracy: 0.9375
Epoch 00231: val_accuracy did not improve from 0.91000
41/41 [==============================] - 0s 996us/step - loss: 0.3728 - accuracy: 0.8993 - val_loss: 0.3601 - val_accuracy: 0.9050
Epoch 232/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3877 - accuracy: 0.8438
Epoch 00232: val_accuracy did not improve from 0.91000
41/41 [==

Epoch 258/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3484 - accuracy: 0.9375
Epoch 00258: val_accuracy did not improve from 0.91500
41/41 [==============================] - 0s 1ms/step - loss: 0.3603 - accuracy: 0.9063 - val_loss: 0.3465 - val_accuracy: 0.9050
Epoch 259/10000
 1/41 [..............................] - ETA: 0s - loss: 0.4128 - accuracy: 0.9375
Epoch 00259: val_accuracy did not improve from 0.91500
41/41 [==============================] - 0s 1ms/step - loss: 0.3595 - accuracy: 0.9063 - val_loss: 0.3459 - val_accuracy: 0.9050
Epoch 260/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3314 - accuracy: 0.9062
Epoch 00260: val_accuracy did not improve from 0.91500
41/41 [==============================] - 0s 997us/step - loss: 0.3590 - accuracy: 0.9040 - val_loss: 0.3462 - val_accuracy: 0.9000
Epoch 261/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3882 - accuracy: 0.8125
Epoch 00261: val_accuracy did not improve from 0.9

Epoch 287/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2976 - accuracy: 0.9688
Epoch 00287: val_accuracy did not improve from 0.91500
41/41 [==============================] - 0s 1ms/step - loss: 0.3481 - accuracy: 0.9133 - val_loss: 0.3340 - val_accuracy: 0.9050
Epoch 288/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3680 - accuracy: 0.9062
Epoch 00288: val_accuracy did not improve from 0.91500
41/41 [==============================] - 0s 999us/step - loss: 0.3470 - accuracy: 0.9102 - val_loss: 0.3342 - val_accuracy: 0.9050
Epoch 289/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2772 - accuracy: 0.9688
Epoch 00289: val_accuracy did not improve from 0.91500
41/41 [==============================] - 0s 991us/step - loss: 0.3472 - accuracy: 0.9173 - val_loss: 0.3329 - val_accuracy: 0.9050
Epoch 290/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2680 - accuracy: 0.9688
Epoch 00290: val_accuracy did not improve from 0

In [359]:
checkpoint_path = "xinshuai_models/linearmodel/8\\0243.ckpt"
linear_model.load_weights(checkpoint_path)

In [360]:
predictions = linear_model.evaluate(valid_set)

7/7 [==============================] - 0s 712us/step - loss: 0.3538 - accuracy: 0.9150


In [361]:
input_size = 16
train_set = get_dataset(train_paths, input_size)
valid_set = get_dataset(test_paths, input_size)
linear_model = train_linear_model(input_size, train_set, valid_set)

Epoch 1/10000
 1/41 [..............................] - ETA: 0s - loss: 1.0186 - accuracy: 0.4688WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0000s vs `on_test_batch_end` time: 0.0010s). Check your callbacks.

Epoch 00001: val_accuracy improved from -inf to 0.55000, saving model to xinshuai_models/linearmodel/16\0001.ckpt
41/41 [==============================] - 0s 7ms/step - loss: 0.6999 - accuracy: 0.5550 - val_loss: 0.6796 - val_accuracy: 0.5500
Epoch 2/10000
 1/41 [..............................] - ETA: 0s - loss: 0.7334 - accuracy: 0.3750
Epoch 00002: val_accuracy improved from 0.55000 to 0.65500, saving model to xinshuai_models/linearmodel/16\0002.ckpt
41/41 [==============================] - 0s 2ms/step - loss: 0.6642 - accuracy: 0.6167 - val_loss: 0.6589 - val_accuracy: 0.6550
Epoch 3/10000
 1/41 [..............................] - ETA: 0s - loss: 0.6609 - accuracy: 0.6250
Epoch 00003: val_accuracy improved from 0.65500

Epoch 27/10000
 1/41 [..............................] - ETA: 0s - loss: 0.4098 - accuracy: 0.9062
Epoch 00027: val_accuracy improved from 0.88000 to 0.88500, saving model to xinshuai_models/linearmodel/16\0027.ckpt
41/41 [==============================] - 0s 2ms/step - loss: 0.4332 - accuracy: 0.8938 - val_loss: 0.4193 - val_accuracy: 0.8850
Epoch 28/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3570 - accuracy: 0.9688
Epoch 00028: val_accuracy improved from 0.88500 to 0.89500, saving model to xinshuai_models/linearmodel/16\0028.ckpt
41/41 [==============================] - 0s 2ms/step - loss: 0.4340 - accuracy: 0.8712 - val_loss: 0.4138 - val_accuracy: 0.8950
Epoch 29/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3828 - accuracy: 0.9375
Epoch 00029: val_accuracy did not improve from 0.89500
41/41 [==============================] - 0s 1ms/step - loss: 0.4250 - accuracy: 0.8899 - val_loss: 0.4117 - val_accuracy: 0.8700
Epoch 30/10000
 1/41 [........

Epoch 55/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3197 - accuracy: 0.9062
Epoch 00055: val_accuracy did not improve from 0.92500
41/41 [==============================] - 0s 973us/step - loss: 0.3469 - accuracy: 0.9110 - val_loss: 0.3270 - val_accuracy: 0.9200
Epoch 56/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3046 - accuracy: 0.9688
Epoch 00056: val_accuracy did not improve from 0.92500
41/41 [==============================] - 0s 985us/step - loss: 0.3465 - accuracy: 0.9040 - val_loss: 0.3280 - val_accuracy: 0.9200
Epoch 57/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3309 - accuracy: 0.9375
Epoch 00057: val_accuracy did not improve from 0.92500
41/41 [==============================] - 0s 972us/step - loss: 0.3499 - accuracy: 0.8993 - val_loss: 0.3243 - val_accuracy: 0.9250
Epoch 58/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5298 - accuracy: 0.7500
Epoch 00058: val_accuracy did not improve from 0.9

Epoch 84/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3175 - accuracy: 0.8750
Epoch 00084: val_accuracy did not improve from 0.93000
41/41 [==============================] - 0s 1ms/step - loss: 0.3031 - accuracy: 0.9173 - val_loss: 0.2847 - val_accuracy: 0.9300
Epoch 85/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3411 - accuracy: 0.8750
Epoch 00085: val_accuracy did not improve from 0.93000
41/41 [==============================] - 0s 1ms/step - loss: 0.3046 - accuracy: 0.9212 - val_loss: 0.2833 - val_accuracy: 0.9300
Epoch 86/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3379 - accuracy: 0.8750
Epoch 00086: val_accuracy did not improve from 0.93000
41/41 [==============================] - 0s 1ms/step - loss: 0.2999 - accuracy: 0.9165 - val_loss: 0.2792 - val_accuracy: 0.9300
Epoch 87/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3597 - accuracy: 0.8750
Epoch 00087: val_accuracy did not improve from 0.93000
4

Epoch 113/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3191 - accuracy: 0.8125
Epoch 00113: val_accuracy did not improve from 0.93000
41/41 [==============================] - 0s 970us/step - loss: 0.2781 - accuracy: 0.9180 - val_loss: 0.2546 - val_accuracy: 0.9250
Epoch 114/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2583 - accuracy: 0.9375
Epoch 00114: val_accuracy did not improve from 0.93000
41/41 [==============================] - 0s 1ms/step - loss: 0.2707 - accuracy: 0.9258 - val_loss: 0.2542 - val_accuracy: 0.9200
Epoch 115/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3028 - accuracy: 0.9062
Epoch 00115: val_accuracy did not improve from 0.93000
41/41 [==============================] - 0s 997us/step - loss: 0.2702 - accuracy: 0.9258 - val_loss: 0.2528 - val_accuracy: 0.9300
Epoch 116/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3211 - accuracy: 0.9062
Epoch 00116: val_accuracy did not improve from 0

Epoch 142/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2293 - accuracy: 0.9375
Epoch 00142: val_accuracy did not improve from 0.94000
41/41 [==============================] - 0s 1ms/step - loss: 0.2500 - accuracy: 0.9360 - val_loss: 0.2352 - val_accuracy: 0.9350
Epoch 143/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2185 - accuracy: 0.9688
Epoch 00143: val_accuracy did not improve from 0.94000
41/41 [==============================] - 0s 1ms/step - loss: 0.2493 - accuracy: 0.9360 - val_loss: 0.2341 - val_accuracy: 0.9350
Epoch 144/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1863 - accuracy: 0.9688
Epoch 00144: val_accuracy did not improve from 0.94000
41/41 [==============================] - 0s 1ms/step - loss: 0.2490 - accuracy: 0.9360 - val_loss: 0.2349 - val_accuracy: 0.9300
Epoch 145/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2420 - accuracy: 0.9375
Epoch 00145: val_accuracy did not improve from 0.940

Epoch 171/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3248 - accuracy: 0.8438
Epoch 00171: val_accuracy improved from 0.94000 to 0.94500, saving model to xinshuai_models/linearmodel/16\0171.ckpt
41/41 [==============================] - 0s 2ms/step - loss: 0.2338 - accuracy: 0.9352 - val_loss: 0.2208 - val_accuracy: 0.9450
Epoch 172/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2839 - accuracy: 0.9062
Epoch 00172: val_accuracy did not improve from 0.94500
41/41 [==============================] - 0s 1ms/step - loss: 0.2364 - accuracy: 0.9368 - val_loss: 0.2275 - val_accuracy: 0.9350
Epoch 173/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1951 - accuracy: 0.9688
Epoch 00173: val_accuracy did not improve from 0.94500
41/41 [==============================] - 0s 1ms/step - loss: 0.2527 - accuracy: 0.9188 - val_loss: 0.2225 - val_accuracy: 0.9350
Epoch 174/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2424 - accurac

Epoch 200/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2217 - accuracy: 0.9688
Epoch 00200: val_accuracy did not improve from 0.94500
41/41 [==============================] - 0s 1ms/step - loss: 0.2210 - accuracy: 0.9407 - val_loss: 0.2184 - val_accuracy: 0.9300
Epoch 201/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1965 - accuracy: 0.9688
Epoch 00201: val_accuracy did not improve from 0.94500
41/41 [==============================] - 0s 1ms/step - loss: 0.2348 - accuracy: 0.9266 - val_loss: 0.2156 - val_accuracy: 0.9350
Epoch 202/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2469 - accuracy: 0.9375
Epoch 00202: val_accuracy did not improve from 0.94500
41/41 [==============================] - 0s 1ms/step - loss: 0.2206 - accuracy: 0.9407 - val_loss: 0.2107 - val_accuracy: 0.9450
Epoch 203/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2272 - accuracy: 0.9062
Epoch 00203: val_accuracy did not improve from 0.945

Epoch 229/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1686 - accuracy: 0.9688
Epoch 00229: val_accuracy did not improve from 0.94500
41/41 [==============================] - 0s 1ms/step - loss: 0.2166 - accuracy: 0.9352 - val_loss: 0.2049 - val_accuracy: 0.9450
Epoch 230/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1880 - accuracy: 0.9688
Epoch 00230: val_accuracy did not improve from 0.94500
41/41 [==============================] - 0s 1ms/step - loss: 0.2135 - accuracy: 0.9415 - val_loss: 0.2046 - val_accuracy: 0.9400
Epoch 231/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3069 - accuracy: 0.9375
Epoch 00231: val_accuracy did not improve from 0.94500
41/41 [==============================] - 0s 1ms/step - loss: 0.2139 - accuracy: 0.9352 - val_loss: 0.2068 - val_accuracy: 0.9300
Epoch 232/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2226 - accuracy: 0.8750
Epoch 00232: val_accuracy did not improve from 0.945

In [362]:
checkpoint_path = "xinshuai_models/linearmodel/16\\0171.ckpt"
linear_model.load_weights(checkpoint_path)
predictions = linear_model.evaluate(valid_set)

7/7 [==============================] - 0s 570us/step - loss: 0.2208 - accuracy: 0.9450


In [363]:
input_size = 32
train_set = get_dataset(train_paths, input_size)
valid_set = get_dataset(test_paths, input_size)
linear_model = train_linear_model(input_size, train_set, valid_set)

Epoch 1/10000
 1/41 [..............................] - ETA: 0s - loss: 0.7843 - accuracy: 0.3438
Epoch 00001: val_accuracy improved from -inf to 0.64000, saving model to xinshuai_models/linearmodel/32\0001.ckpt
41/41 [==============================] - 0s 6ms/step - loss: 0.6939 - accuracy: 0.5441 - val_loss: 0.6293 - val_accuracy: 0.6400
Epoch 2/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5947 - accuracy: 0.7500
Epoch 00002: val_accuracy improved from 0.64000 to 0.71000, saving model to xinshuai_models/linearmodel/32\0002.ckpt
41/41 [==============================] - 0s 2ms/step - loss: 0.6211 - accuracy: 0.6542 - val_loss: 0.5854 - val_accuracy: 0.7100
Epoch 3/10000
 1/41 [..............................] - ETA: 0s - loss: 0.5962 - accuracy: 0.7188
Epoch 00003: val_accuracy improved from 0.71000 to 0.77500, saving model to xinshuai_models/linearmodel/32\0003.ckpt
41/41 [==============================] - 0s 2ms/step - loss: 0.5874 - accuracy: 0.7057 - val_loss: 0.54

Epoch 28/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2030 - accuracy: 0.9375
Epoch 00028: val_accuracy improved from 0.93500 to 0.94000, saving model to xinshuai_models/linearmodel/32\0028.ckpt
41/41 [==============================] - 0s 2ms/step - loss: 0.3003 - accuracy: 0.9079 - val_loss: 0.2679 - val_accuracy: 0.9400
Epoch 29/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2389 - accuracy: 0.9375
Epoch 00029: val_accuracy did not improve from 0.94000
41/41 [==============================] - 0s 1ms/step - loss: 0.2991 - accuracy: 0.9079 - val_loss: 0.2624 - val_accuracy: 0.9350
Epoch 30/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2372 - accuracy: 0.9688
Epoch 00030: val_accuracy did not improve from 0.94000
41/41 [==============================] - 0s 1ms/step - loss: 0.2832 - accuracy: 0.9204 - val_loss: 0.2657 - val_accuracy: 0.9150
Epoch 31/10000
 1/41 [..............................] - ETA: 0s - loss: 0.3503 - accuracy: 0

Epoch 57/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1373 - accuracy: 1.0000
Epoch 00057: val_accuracy did not improve from 0.94500
41/41 [==============================] - 0s 1ms/step - loss: 0.2634 - accuracy: 0.9110 - val_loss: 0.2091 - val_accuracy: 0.9350
Epoch 58/10000
 1/41 [..............................] - ETA: 0s - loss: 0.4513 - accuracy: 0.8750
Epoch 00058: val_accuracy did not improve from 0.94500
41/41 [==============================] - 0s 1ms/step - loss: 0.2175 - accuracy: 0.9454 - val_loss: 0.2037 - val_accuracy: 0.9300
Epoch 59/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2369 - accuracy: 0.9375
Epoch 00059: val_accuracy did not improve from 0.94500
41/41 [==============================] - 0s 1ms/step - loss: 0.2122 - accuracy: 0.9415 - val_loss: 0.2118 - val_accuracy: 0.9250
Epoch 60/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1354 - accuracy: 1.0000
Epoch 00060: val_accuracy did not improve from 0.94500
4

Epoch 86/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2322 - accuracy: 0.9062
Epoch 00086: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 1ms/step - loss: 0.1865 - accuracy: 0.9469 - val_loss: 0.1846 - val_accuracy: 0.9350
Epoch 87/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1980 - accuracy: 0.9375
Epoch 00087: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 1ms/step - loss: 0.1824 - accuracy: 0.9524 - val_loss: 0.1934 - val_accuracy: 0.9350
Epoch 88/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1919 - accuracy: 0.9062
Epoch 00088: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 1ms/step - loss: 0.1863 - accuracy: 0.9422 - val_loss: 0.1977 - val_accuracy: 0.9450
Epoch 89/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2231 - accuracy: 0.8750
Epoch 00089: val_accuracy did not improve from 0.95000
4

Epoch 115/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1582 - accuracy: 0.9688
Epoch 00115: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 1ms/step - loss: 0.1714 - accuracy: 0.9477 - val_loss: 0.1858 - val_accuracy: 0.9400
Epoch 116/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1852 - accuracy: 0.9688
Epoch 00116: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 1ms/step - loss: 0.1633 - accuracy: 0.9571 - val_loss: 0.1752 - val_accuracy: 0.9400
Epoch 117/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1612 - accuracy: 0.9375
Epoch 00117: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 1ms/step - loss: 0.1606 - accuracy: 0.9532 - val_loss: 0.1841 - val_accuracy: 0.9400
Epoch 118/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1115 - accuracy: 1.0000
Epoch 00118: val_accuracy did not improve from 0.950

In [364]:
checkpoint_path = "xinshuai_models/linearmodel/32\\0062.ckpt"
linear_model.load_weights(checkpoint_path)
predictions = linear_model.evaluate(valid_set)

7/7 [==============================] - 0s 999us/step - loss: 0.2097 - accuracy: 0.9500


In [365]:
input_size = 64
train_set = get_dataset(train_paths, input_size)
valid_set = get_dataset(test_paths, input_size)
linear_model = train_linear_model(input_size, train_set, valid_set)

Epoch 1/10000
 1/41 [..............................] - ETA: 0s - loss: 0.7957 - accuracy: 0.5312
Epoch 00001: val_accuracy improved from -inf to 0.51000, saving model to xinshuai_models/linearmodel/64\0001.ckpt
41/41 [==============================] - 0s 6ms/step - loss: 0.7811 - accuracy: 0.5550 - val_loss: 0.7786 - val_accuracy: 0.5100
Epoch 2/10000
 1/41 [..............................] - ETA: 0s - loss: 1.0238 - accuracy: 0.2812
Epoch 00002: val_accuracy improved from 0.51000 to 0.61000, saving model to xinshuai_models/linearmodel/64\0002.ckpt
41/41 [==============================] - 0s 2ms/step - loss: 0.6016 - accuracy: 0.6518 - val_loss: 0.6046 - val_accuracy: 0.6100
Epoch 3/10000
 1/41 [..............................] - ETA: 0s - loss: 0.7237 - accuracy: 0.5312
Epoch 00003: val_accuracy improved from 0.61000 to 0.74000, saving model to xinshuai_models/linearmodel/64\0003.ckpt
41/41 [==============================] - 0s 2ms/step - loss: 0.5383 - accuracy: 0.7119 - val_loss: 0.46

Epoch 28/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1020 - accuracy: 0.9688
Epoch 00028: val_accuracy did not improve from 0.94000
41/41 [==============================] - 0s 2ms/step - loss: 0.1993 - accuracy: 0.9407 - val_loss: 0.1901 - val_accuracy: 0.9350
Epoch 29/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1302 - accuracy: 1.0000
Epoch 00029: val_accuracy did not improve from 0.94000
41/41 [==============================] - 0s 1ms/step - loss: 0.2036 - accuracy: 0.9368 - val_loss: 0.2632 - val_accuracy: 0.8750
Epoch 30/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2051 - accuracy: 0.9062
Epoch 00030: val_accuracy did not improve from 0.94000
41/41 [==============================] - 0s 2ms/step - loss: 0.1986 - accuracy: 0.9344 - val_loss: 0.1879 - val_accuracy: 0.9350
Epoch 31/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1934 - accuracy: 0.9062
Epoch 00031: val_accuracy did not improve from 0.94000
4

Epoch 57/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2646 - accuracy: 0.8750
Epoch 00057: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 1ms/step - loss: 0.3677 - accuracy: 0.8595 - val_loss: 0.1689 - val_accuracy: 0.9300
Epoch 58/10000
 1/41 [..............................] - ETA: 0s - loss: 0.2902 - accuracy: 0.9375
Epoch 00058: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 2ms/step - loss: 0.1401 - accuracy: 0.9586 - val_loss: 0.1661 - val_accuracy: 0.9350
Epoch 59/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1171 - accuracy: 0.9375
Epoch 00059: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 2ms/step - loss: 0.1354 - accuracy: 0.9602 - val_loss: 0.1761 - val_accuracy: 0.9400
Epoch 60/10000
 1/41 [..............................] - ETA: 0s - loss: 0.0761 - accuracy: 1.0000
Epoch 00060: val_accuracy did not improve from 0.95000
4

 1/41 [..............................] - ETA: 0s - loss: 0.1230 - accuracy: 0.9688
Epoch 00086: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 1ms/step - loss: 0.1066 - accuracy: 0.9688 - val_loss: 0.1736 - val_accuracy: 0.9300
Epoch 87/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1189 - accuracy: 1.0000
Epoch 00087: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 1ms/step - loss: 0.1084 - accuracy: 0.9703 - val_loss: 0.1755 - val_accuracy: 0.9500
Epoch 88/10000
 1/41 [..............................] - ETA: 0s - loss: 0.0830 - accuracy: 0.9688
Epoch 00088: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 1ms/step - loss: 0.1123 - accuracy: 0.9649 - val_loss: 0.1811 - val_accuracy: 0.9400
Epoch 89/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1370 - accuracy: 0.9375
Epoch 00089: val_accuracy did not improve from 0.95000
41/41 [=========

 1/41 [..............................] - ETA: 0s - loss: 0.1129 - accuracy: 0.9688
Epoch 00115: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 2ms/step - loss: 0.1105 - accuracy: 0.9571 - val_loss: 0.1866 - val_accuracy: 0.9400
Epoch 116/10000
 1/41 [..............................] - ETA: 0s - loss: 0.1395 - accuracy: 0.9375
Epoch 00116: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 2ms/step - loss: 0.0908 - accuracy: 0.9766 - val_loss: 0.1818 - val_accuracy: 0.9450
Epoch 117/10000
 1/41 [..............................] - ETA: 0s - loss: 0.0848 - accuracy: 0.9688
Epoch 00117: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 1ms/step - loss: 0.0922 - accuracy: 0.9742 - val_loss: 0.1971 - val_accuracy: 0.9400
Epoch 118/10000
 1/41 [..............................] - ETA: 0s - loss: 0.0601 - accuracy: 1.0000
Epoch 00118: val_accuracy did not improve from 0.95000
41/41 [======

In [368]:
checkpoint_path = "xinshuai_models/linearmodel/64\\0054.ckpt"
linear_model.load_weights(checkpoint_path)
predictions = linear_model.evaluate(valid_set)

7/7 [==============================] - 0s 1ms/step - loss: 0.1688 - accuracy: 0.9500


In [369]:
input_size = 128
train_set = get_dataset(train_paths, input_size)
valid_set = get_dataset(test_paths, input_size)
linear_model = train_linear_model(input_size, train_set, valid_set)

Epoch 1/10000
26/41 [==================>...........] - ETA: 0s - loss: 1.4599 - accuracy: 0.5409
Epoch 00001: val_accuracy improved from -inf to 0.56000, saving model to xinshuai_models/linearmodel/128\0001.ckpt
41/41 [==============================] - 0s 8ms/step - loss: 1.1477 - accuracy: 0.5925 - val_loss: 0.6821 - val_accuracy: 0.5600
Epoch 2/10000
19/41 [============>.................] - ETA: 0s - loss: 0.4991 - accuracy: 0.7697
Epoch 00002: val_accuracy did not improve from 0.56000
41/41 [==============================] - 0s 3ms/step - loss: 0.4619 - accuracy: 0.7970 - val_loss: 2.7070 - val_accuracy: 0.4900
Epoch 3/10000
21/41 [==============>...............] - ETA: 0s - loss: 1.9125 - accuracy: 0.5372
Epoch 00003: val_accuracy improved from 0.56000 to 0.70500, saving model to xinshuai_models/linearmodel/128\0003.ckpt
41/41 [==============================] - 0s 3ms/step - loss: 1.2555 - accuracy: 0.6347 - val_loss: 0.5664 - val_accuracy: 0.7050
Epoch 4/10000
19/41 [============>

Epoch 57/10000
24/41 [================>.............] - ETA: 0s - loss: 1.3055 - accuracy: 0.6940
Epoch 00057: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 3ms/step - loss: 0.9558 - accuracy: 0.7525 - val_loss: 0.2601 - val_accuracy: 0.9250
Epoch 58/10000
24/41 [================>.............] - ETA: 0s - loss: 0.1314 - accuracy: 0.9492
Epoch 00058: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 3ms/step - loss: 0.1121 - accuracy: 0.9578 - val_loss: 0.2142 - val_accuracy: 0.9400
Epoch 59/10000
28/41 [===================>..........] - ETA: 0s - loss: 0.0897 - accuracy: 0.9643
Epoch 00059: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 2ms/step - loss: 0.0992 - accuracy: 0.9625 - val_loss: 0.1995 - val_accuracy: 0.9450
Epoch 60/10000
25/41 [=================>............] - ETA: 0s - loss: 0.0845 - accuracy: 0.9700
Epoch 00060: val_accuracy did not improve from 0.95000
4

22/41 [===============>..............] - ETA: 0s - loss: 0.0746 - accuracy: 0.9759
Epoch 00086: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 3ms/step - loss: 0.0702 - accuracy: 0.9750 - val_loss: 0.2420 - val_accuracy: 0.9450
Epoch 87/10000
24/41 [================>.............] - ETA: 0s - loss: 0.0836 - accuracy: 0.9714
Epoch 00087: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 3ms/step - loss: 0.1709 - accuracy: 0.9336 - val_loss: 0.4012 - val_accuracy: 0.8750
Epoch 88/10000
31/41 [=====================>........] - ETA: 0s - loss: 0.0671 - accuracy: 0.9778
Epoch 00088: val_accuracy did not improve from 0.95000
41/41 [==============================] - 0s 4ms/step - loss: 0.0672 - accuracy: 0.9766 - val_loss: 0.2450 - val_accuracy: 0.9450
Epoch 89/10000
39/41 [===========================>..] - ETA: 0s - loss: 0.1166 - accuracy: 0.9519
Epoch 00089: val_accuracy did not improve from 0.95000
41/41 [=========

In [370]:
checkpoint_path = "xinshuai_models/linearmodel/128\\0026.ckpt"
linear_model.load_weights(checkpoint_path)
predictions = linear_model.evaluate(valid_set)

7/7 [==============================] - 0s 2ms/step - loss: 0.1663 - accuracy: 0.9500


In [371]:
df = pd.DataFrame({"input_sizes":[8, 16, 32, 64, 128],
                   "accuracys":[0.9150, 0.9450, 0.9500, 0.9500, 0.9500]})
df.to_csv("accuracy_in_different_input_size")